# Running on a dataset

We will use nipype for this.

## Imports

In [32]:
import os
import os.path as op
import glob
from pathlib import Path
from ndmg.scripts import ndmg_dwi_pipeline
from ndmg.utils import s3_utils

## Grab input dataset

In [40]:
# Download test data
data_dir = op.expanduser("~") + "/.ndmg/HNU1t"
if not op.isdir(data_dir):
    os.mkdir(data_dir)
    s3_utils.s3_get_data('ndmg-data', 'HNU1t', data_dir, force=True)

# Specify base directory and paths to input files (dwi, bvecs, bvals, and t1w required)
input_files = []
subs = [i.split('/')[-1] for i in glob.glob(data_dir + '/sub*')]
for subject in subs:
    t1w = data_dir + '/' +  subject + '/ses-1/anat/' + subject + '_ses-1_T1w.nii.gz'
    bvals = data_dir + '/' + subject + '/ses-1/dwi/' + subject + '_ses-1_dwi.bval'
    bvecs = data_dir + '/' + subject + '/ses-1/dwi/' + subject + '_ses-1_dwi.bvec'
    dwi = data_dir + '/' + subject + '/ses-1/dwi/' + subject + '_ses-1_dwi.nii.gz'
    input_files.append((subject, t1w, bvals, bvecs, dwi))
    
    

## Set up atlas and parameters

In [40]:
###USER-SPECIFIED PARAMETERS###
# Specify NDMG run parameters
## CHOICES:
## atlases = ['desikan', 'CPAC200', 'DKT', 'HarvardOxfordcort', 'HarvardOxfordsub', 'JHU', 'Schaefer2018-200', 'Talairach', 'aal', 'brodmann', 'glasser', 'yeo-7-liberal', 'yeo-17-liberal']
## mod_types = ['det', 'prob']
## track_types = ['local', 'particle']
## mods = ['csa', 'csd']
## regs = ['native', 'native_dsn', 'mni']
## vox_size = ['1mm', '2mm']

# Select an atlas
atlas = 'desikan'
mod_type = 'det'
track_type = 'local'
mod_func = 'csd'
reg_style = 'native'
vox_size = '2mm'
seeds = 20

# Set an output directory
outdir = '/tmp/output_{}_{}_{}_{}_{}_{}_{}'.format(atlas, mod_type, track_type, mod_func, seeds, reg_style, subject)
###############################
# Download neuroparc files
atlas_dir = op.expanduser("~") + "/.ndmg/ndmg_atlases"

# Requires that git lfs is installed
if not op.isdir(atlas_dir):
    os.mkdir(atlas_dir)
    clone = "https://github.com/neurodata/neuroparc.git"
    os.system("git lfs clone {} {}".format(clone, atlas_dir))

# Auto-set paths to neuroparc files
mask = str(Path.home()) + '/.ndmg/ndmg_atlases/atlases/mask/MNI152NLin6_res-2x2x2_T1w_descr-brainmask.nii.gz'
labels = [i for i in glob.glob(str(Path.home()) + '/.ndmg/ndmg_atlases/atlases/label/Human/*2x2x2.nii.gz') if atlas in i]

## Nipype imports

In [41]:
'''
Import nipype utilities (must install nipype)
'''
from nipype.pipeline import engine as pe
from nipype.interfaces import utility as niu
from nipype.interfaces.base import BaseInterface, BaseInterfaceInputSpec, TraitedSpec, File, traits, SimpleInterface

## Nipype workflow

### Setup

In [42]:
'''
Construct multi-subject workflow
'''

wf = pe.Workflow(name="NDMG_group_wf")

# Create an import list of modules/objects to permeate the workflow environment
import_list = ["import shutil", "import time", "import os", "import numpy as np", "import networkx as nx",
               "import nibabel as nib", "import warnings", "warnings.filterwarnings(\"ignore\")",
               "np.warnings.filterwarnings(\"ignore\")", "warnings.simplefilter(\"ignore\")", 
               "from subprocess import Popen", "from dipy.tracking.streamline import Streamlines",
              "import ndmg", "from ndmg import preproc as mgp", "from ndmg.utils import gen_utils as mgu", 
               "from ndmg.utils import s3_utils", "from ndmg.register import gen_reg as mgr", 
               "from ndmg.track import gen_track as mgt", "from ndmg.graph import gen_graph as mgg",
              "from ndmg.utils.bids_utils import name_resource", "from ndmg.stats.qa_tensor import *", 
               "from ndmg.stats.qa_fibers import *", "from datetime import datetime"]

inputnode = pe.Node(niu.IdentityInterface(fields=['atlas', 'mask', 'labels', 
                                                  'outdir', 'vox_size', 'mod_type', 
                                                  'track_type', 'mod_func', 'reg_style', 'clean', 
                                                  'skedy', 'skipreg']),
                    name='inputnode', imports=import_list)

inputnode.inputs.atlas = atlas
inputnode.inputs.mask = mask
inputnode.inputs.labels = labels[0]
inputnode.inputs.outdir = outdir
inputnode.inputs.vox_size = vox_size
inputnode.inputs.mod_type = mod_type
inputnode.inputs.track_type = track_type
inputnode.inputs.mod_func = mod_func
inputnode.inputs.seeds = seeds
inputnode.inputs.reg_style = reg_style
inputnode.inputs.clean = False
inputnode.inputs.skedy = False
inputnode.inputs.skipreg = False

# Make NDMG an interface
class ndmgDWIworkerInputSpec(BaseInterfaceInputSpec):
    """
    Input interface wrapper for ndmgDWIworker
    """
    dwi = traits.Str(mandatory=True)
    bvals = traits.Str(mandatory=True)
    bvecs = traits.Str(mandatory=True)
    t1w = traits.Str(mandatory=True)
    atlas = traits.Str(mandatory=True)
    mask = traits.Str(mandatory=True)
    labels = traits.Str(mandatory=True)
    outdir = traits.Str(mandatory=True)
    vox_size = traits.Str(mandatory=True)
    mod_type = traits.Str(mandatory=True)
    track_type = traits.Str(mandatory=True)
    mod_func = traits.Str(mandatory=True)
    seeds = traits.Str(mandatory=True)
    reg_style = traits.Str(mandatory=True)
    clean = traits.Bool(mandatory=False)
    skedy = traits.Bool(mandatory=False)
    skipreg = traits.Bool(mandatory=False)

class ndmgDWIworker(BaseInterface):
    """
    Interface wrapper for ndmgDWIworker
    """
    input_spec = ndmgDWIworkerInputSpec

    def _run_interface(self, runtime):
        out = ndmg_dwi_pipeline.ndmg_dwi_worker(
            self.inputs.dwi,
            self.inputs.bvals,
            self.inputs.bvecs,
            self.inputs.t1w,
            self.inputs.atlas,
            self.inputs.mask,
            self.inputs.labels,
            self.inputs.outdir,
            self.inputs.vox_size,
            self.inputs.mod_type,
            self.inputs.track_type,
            self.inputs.mod_func,
            self.inputs.seeds,
            self.inputs.reg_style,
            self.inputs.clean,
            self.inputs.skedy,
            self.inputs.skipreg)
        setattr(self, '_outpath', out)
        return runtime

### Set up the main pipeline as a nipype node

In [42]:
# Make the NDMG wf itself a node of the DAG
ndmg_dwi_worker_node = pe.Node(interface=ndmgDWIworker(),
                                  inputs=['dwi', 'bvals', 'bvecs', 't1w',
                                          'atlas', 'mask', 'labels', 
                                          'outdir', 'vox_size', 'mod_type', 
                                          'track_type', 'mod_func', 'seeds', 
                                          'reg_style', 'clean', 'skedy', 'skipreg'],
                                   nested=True, synchronize=True, name='ndmg_dwi_worker_node', imports=import_list)

# Restrict cpu and memory for that node such that scheduling will wait for available 
# resources before proceeding with additional subjects in parallel
ndmg_dwi_worker_node.interface.n_procs = 2
ndmg_dwi_worker_node.interface.mem_gb = 8

Here, we define all of the variables that change depending on the file as node iterables.

In [42]:
# Make subject files iterable
ndmg_dwi_worker_node.iterables = [("dwi", [i[4] for i in input_files]), ("bvals", [i[2] for i in input_files]), ("bvecs", [i[3] for i in input_files]), ("t1w", [i[1] for i in input_files])]

# Connect variables of workflow
wf.connect([
    (inputnode, ndmg_dwi_worker_node, [('atlas', 'atlas'), 
                                       ('mask', 'mask'), 
                                       ('labels', 'labels'), 
                                       ('outdir', 'outdir'), 
                                       ('vox_size', 'vox_size'), 
                                       ('mod_type', 'mod_type'), 
                                       ('track_type', 'track_type'), 
                                       ('mod_func', 'mod_func'), 
                                       ('seeds', 'seeds'), 
                                       ('reg_style', 'reg_style'), 
                                       ('clean', 'clean'), 
                                       ('skedy', 'skedy'), 
                                       ('skipreg', 'skipreg')])
])

# Configure wf execution
cfg = dict(execution={'stop_on_first_crash': True, 'hash_method': 'content', 'crashfile_format': 'txt',
                      'display_variable': ':0', 'job_finished_timeout': 65, 'matplotlib_backend': 'Agg',
                      'use_relative_paths': True, 'parameterize_dirs': False,
                      'remove_unnecessary_outputs': False, 'remove_node_directories': False,
                      'raise_insufficient': True})
for key in cfg.keys():
    for setting, value in cfg[key].items():
        wf.config[key][setting] = value

## Run nipype

In [42]:
# Set runtime directory to home directory
wf.base_dir = op.expanduser("~")

# Run wf using multiproc scheduler
wf.run(plugin='MultiProc', plugin_args = {'n_procs': int(2), 'memory_gb': int(16)})